In [24]:
import sys
import os
import bz2
import csv
import pandas as pd
import ast
import ujson
import numpy as np

import datashader as ds
import datashader.transfer_functions as tf

ENCODING = "UTF-8"

# dirpath = "Data/OTCamera/"
# filename = "Testvideo_Cars-Cyclist_FR20_2020-01-01_00-00-00"
dirpath = "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC07_23-09-19_17-00-00/"
filename = "OTCamera07_FR20_2023-09-19_17-00-00_Sued"
fileending = '.ottrk'
filepath = os.path.join(dirpath, filename) + fileending

In [ ]:
def ottrk_to_txt(filepath, key1 = 'data', key2 = 'detections'):
    # Open ottrk-file
    with bz2.open(filepath, "rt", encoding=ENCODING) as file:
        dictfile = ujson.load(file)
    
    # Write in DataFrame
    detections = pd.DataFrame.from_dict(dictfile[key1][key2])
    detections = detections[['x', 'y', 'w', 'h', 'frame', 'track-id', 'class']] #bb_left = x, bb_top = y

    # Transform to MOTChallenge-format (x,y,z are ignored fpr 2D-challenges)
    MOTdetections = detections
    MOTdetections['conf'] = 1
    MOTdetections['3D_x'] = -1
    MOTdetections['3D_y'] = -1
    MOTdetections['3D_z'] = -1
    MOTdetections = MOTdetections[['frame', 'track-id', 'x', 'y', 'h', 'w', 'conf', '3D_x', '3D_y', '3D_z']]

    # Export
    # MOTdetections.to_csv(dirpath + filename + '.txt', encoding='utf-8', header=False, index=False)

    return detections, MOTdetections

OTCdetections, MOTdetections = ottrk_to_txt(filepath = filepath)

OTCdetections['sec'] = (OTCdetections['frame'] - OTCdetections['frame'].min()) / 20 #20fps


In [31]:
# Frames filtern

# data = detections[detections['frame'] >=3380]
# data = data[data['frame'] <=5000]
OTCdetections = OTCdetections[OTCdetections['class'] == 'pedestrian']

print(OTCdetections.groupby(['track-id'])['track-id'].count())
# print(data['track-id'].value_counts())
# data = data[data['track-id'] == 14476]

track-id
12      107
20        5
22      262
26        8
30        2
       ... 
1229      9
1231      1
1236      7
1241      2
1246     60
Name: track-id, Length: 300, dtype: int64


## Plotting

In [19]:

plotdata = data
plotdata['x'] = plotdata['x'] * -1
plotdata['y'] = plotdata['y']
# Default plot ranges:
x_range = (plotdata['x'].min(), plotdata['x'].max())
y_range = (plotdata['y'].min(), plotdata['y'].max())
print(x_range)
print(y_range)
# x_range = (0, 1000)
# y_range = (-1000, 0)
plotdata = plotdata.sort_values(by=['track-id', 'frame'])

# ids = plotdata.groupby('track-id')

# for id in ids:
#     print(id)

def create_image(w=500, h=500):
    cvs = ds.Canvas(x_range=x_range, y_range=y_range, plot_height=h, plot_width=w)
    agg = cvs.line(plotdata, 'x', 'y', agg=ds.any())
    return tf.shade(agg, cmap = 'white')

# create_image()

(10.378253936767578, 638.6010131835938)
(80.53295135498047, 464.22369384765625)


In [58]:
## OTEvents

eventscsv = pd.read_csv(dirpath + 'events.csv')
eventscsv = eventscsv[eventscsv['road_user_type'] == 'pedestrian']

def events_to_txt(filepath, key1 = 'sections', key2 = 'event_list'):
    # Open ottrk-file
    with bz2.open(filepath, "rt", encoding=ENCODING) as file:
        dictfile = ujson.load(file)

    # Write in DataFrame
    sections = pd.DataFrame.from_dict(dictfile['sections'])
    events = pd.DataFrame.from_dict(dictfile['event_list'])
    # detections = detections[['x', 'y', 'w', 'h', 'frame', 'track-id', 'class']] #bb_left = x, bb_top = y


    # Export
    # MOTdetections.to_csv(dirpath + filename + '.txt', encoding='utf-8', header=False, index=False)

    return sections, events
sections, events = events_to_txt(filepath = dirpath + 'events.otevents')

events = events[events['road_user_type'] == 'pedestrian']